---



# Unit 2 - Part 3a: Chain of Thought (CoT)

## 1. Introduction: The Inner Monologue

Standard LLMs try to jump straight to the answer. For complex problems (math, logic), this often fails.

**Chain of Thought (CoT)** forces the model to "think out loud" before answering. 

### Why use a "Dumb" Model?
For this unit, we will use **Llama3.1-8b** (via Groq). It is a smaller, faster model.
Why? Because huge models (like Gemini Pro or GPT-4) are often *too smart*—they solve logic riddles instantly without thinking.

To really see the power of Prompt Engineering, we need a model that **needs help**.

### Visualizing the Process (Flowchart)
```mermaid
graph TD
    Input[Question: 5+5*2?]
    Input -->|Standard| Wrong[Answer: 20 (Wrong)]
    Input -->|CoT| Step1[Step 1: 5*2=10]
    Step1 --> Step2[Step 2: 5+10=15]
    Step2 --> Correct[Answer: 15 (Correct)]
```

## 2. Concept: Latent Reasoning

Why does this work?
Because LLMs are "Next Token Predictors".
- If you force it to answer immediately, it must predict the digits `1` and `5` immediately.
- If you let it "think", it generates intermediate tokens (`5`, `*`, `2`, `=`, `1`, `0`).
- The model then **ATTENDS** to these new tokens to compute the final answer.

**Writing is Thinking.**

In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)


[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


## 3. The Experiment: A Tricky Math Problem

Let's try a problem that requires multi-step logic.

**Problem:**
"Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

In [2]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


### Critique
Smaller models often latch onto the visible numbers (5 and 2) and simply add them (7), ignoring the multiplication step implied by "cans".

Let's force it to think.

In [3]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


## 4. Analysis

Look at the output. By explicitly breaking it down:
1.  "Roger starts with 5."
2.  "2 cans * 3 balls = 6 balls."
3.  "5 + 6 = 11."

The model effectively "debugs" its own logic by generating the intermediate steps.

---



# Unit 2 - Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts (GoT)

## 1. Introduction: Beyond A -> B

CoT is linear. But complex reasoning is often nonlinear. We need to explore branches (ToT) or even combine ideas (GoT).

We continue using **Llama3.1-8b via Groq** to show how structure improves performance.

In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed


[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


## 2. Tree of Thoughts (ToT)

ToT explores multiple branches before making a decision. 
**Analogy:** A chess player considering 3 possible moves before playing one.

### Implementation
We will generate 3 distinct solutions for a problem and then use a "Judge" to pick the best one.

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'
    
    1: {sol1}
    2: {sol2}
    3: {sol3}
    
    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]}) 
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a child psychologist, I would recommend **Solution 2: Involve your child in the gardening process** as the most sustainable approach to encourage your 5-year-old to eat vegetables. This approach is not bribery, and it has several benefits that make it an effective and long-lasting solution.

**Why it's sustainable:**

1. **Educational value:** By involving your child in the gardening process, you're teaching them about where food comes from, how it grows, and the process of nurturing and harvesting. This knowledge can help them develop a deeper appreciation for the food they eat.
2. **Emotional connection:** When children are involved in the process of growing their own food, they develop an emotional connection with the vegetables. They become invested in the process and feel a sense of pride and ownership when they harvest the vegetables.
3. **Increased autonomy:** By letting your child help with watering, weeding, and harvesting, you're givin

## 3. Graph of Thoughts (GoT)

You asked: **"Where is Graph of Thoughts?"**

GoT is more complex. It's a network. Information can split, process specific parts, and then **AGGREGATE** back together.

### The Workflow (Writer's Room)
1.  **Split:** Generate 3 independent story plots (Sci-Fi, Fantasy, Mystery).
2.  **Aggregate:** The model reads all 3 and creates a "Master Plot" that combines the best elements of each.
3.  **Refine:** Polish the Master Plot.

```mermaid
graph LR
   Start(Concept) --> A[Draft 1]
   Start --> B[Draft 2]
   Start --> C[Draft 3]
   A & B & C --> Mixer[Aggregator]
   Mixer --> Final[Final Story]
```

In [6]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}
    
    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]}) 
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
"Echoes of Eternity" is a heart-pounding, mind-bending thriller that combines the thrill of science fiction, the passion of romance, and the terror of horror. When brilliant physicist Emma Taylor discovers a way to harness the power of quantum entanglements, she unwittingly sends her younger self back in time to the night her high school sweetheart, Matt, died in a tragic accident. As Emma navigates the treacherous consequences of altering the past, she must confront the dark force that drove Matt's death: a malevolent entity that has been awakened by her time traveling. With each attempt to correct the timeline, Emma finds herself reliving the same fateful night, but with a twist: every time, Matt is still alive, and their love is rekindled - but at a terrible cost, as the entity closes in, threatening to destroy not only her future, but the very fabric of time itself.


## 4. Summary & Comparison Table

| Method | Structure | Best For... | Cost/Latency |
|--------|-----------|-------------|--------------|
| **Simple Prompt** | Input -> Output | Simple facts, summaries | ⭐ Low |
| **CoT (Chain)** | Input -> Steps -> Output | Math, Logic, Debugging | ⭐⭐ Med |
| **ToT (Tree)** | Input -> 3x Branches -> Select -> Output | Strategic decisions, Brainstorming | ⭐⭐⭐ High | 
| **GoT (Graph)** | Input -> Branch -> Mix/Aggregate -> Output | Creative Writing, Research Synthesis | ⭐⭐⭐⭐ V. High |

**Recommendation:** Start with CoT. Only use ToT/GoT if CoT fails.

In [7]:
advanced_prompt = """
You are an expert problem-solving assistant.

TASK:
A student scored the following marks:

Math = 78
Physics = 85
Chemistry = 91

INSTRUCTIONS:
1. Calculate the average.
2. Determine the grade using:
   - A: 90+
   - B: 75–89
   - C: 60–74
   - D: below 60
3. Explain the steps clearly.
4. Return the final answer in structured format.

OUTPUT FORMAT:

Step-by-step reasoning:
<steps>

Final Answer:
Average = ?
Grade = ?
"""

print(llm.invoke(advanced_prompt).content)

**Step-by-step reasoning:**

1. Calculate the average:
   - To find the average, we need to add up all the scores and divide by the number of subjects.
   - Math score = 78
   - Physics score = 85
   - Chemistry score = 91
   - Total score = 78 + 85 + 91 = 254
   - Number of subjects = 3
   - Average = Total score / Number of subjects
   - Average = 254 / 3
   - Average = 84.67

2. Determine the grade:
   - We will compare each subject score with the given grade ranges.
   - Math score = 78 (Below 90, so it can be either C or B, but since 75-89 is B, it will fall under B)
   - Physics score = 85 (Below 90, so it will fall under B)
   - Chemistry score = 91 (Above 90, so it will fall under A)
   - Since the highest grade is 'A', the overall grade will be A

**Final Answer:**
Average = 84.67
Grade = A


In [8]:
# LCLL VERSION 


from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_template("""
You are an expert academic evaluator.

Calculate the average and assign a grade.

Marks:
Math = {math}
Physics = {physics}
Chemistry = {chemistry}

Explain steps and return structured output.
""")

chain = template | llm | StrOutputParser()

print(chain.invoke({
    "math": 78,
    "physics": 85,
    "chemistry": 91
}))

**Evaluation Report**

**Student Information:** 
(No student information is provided, so we will refer to the student as 'Student-X')

**Subjects:** 
- Math
- Physics
- Chemistry

**Marks:**

| Subject | Marks |
|---------|-------|
| Math    | 78    |
| Physics | 85    |
| Chemistry| 91    |

**Calculation Steps:**

1. Add up all the marks: 78 + 85 + 91 = 254
2. Count the number of subjects: 3
3. Calculate the average by dividing the total marks by the number of subjects: 254 ÷ 3 = 84.67

**Structured Output:**

| **Subject** | **Marks** | **Average** |
|-------------|----------|-------------|
| Math        | 78       |             |
| Physics     | 85       |             |
| Chemistry   | 91       |             |
| **Average** |          | 84.67       |

**Grade Assignment:**

Based on the average, a grade can be assigned as follows:

- A: 90-100 (Excellent)
- B: 80-89 (Good)
- C: 70-79 (Fair)
- D: 60-69 (Pass)
- F: Below 60 (Fail)

Given the average of 84.67, the student can be assig